In [ ]:
import os, subprocess

def run(cmd):
    """Helper to execute shell commands with logging."""
    print(f"▶ {cmd}")
    subprocess.run(cmd, shell=True, check=True)

# --- Kaggle setup ---
if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
    run("mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json")

# --- Kaggle data download (only if missing) ---
if not os.path.exists("data/train"):
    run("kaggle competitions download -c aml-competition -p data")
    run("unzip -qo data/aml-competition.zip -d data")

# --- Clone repositories ---
if not os.path.exists("challenge"):
    run("git clone https://github.com/Mamiglia/challenge.git")
if not os.path.exists("CrossFlow"):
    run("git clone https://github.com/qihao067/CrossFlow.git")

# --- Install dependencies ---
!pip install -q torch torchvision torchaudio
!pip install -q openai-clip scikit-learn opencv-python torchdiffeq \
    beautifulsoup4 open_clip_torch scikit-image cython matplotlib accelerate \
    absl-py ml_collections einops wandb ftfy transformers timm tensorboard pycocotools

▶ mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
▶ kaggle competitions download -c aml-competition -p data
▶ unzip -qo data/aml-competition.zip -d data
▶ git clone https://github.com/Mamiglia/challenge.git
▶ git clone https://github.com/qihao067/CrossFlow.git
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from pathlib import Path
from tqdm import tqdm
import sys, random, logging

# Extend path to local repositories
sys.path.extend(["CrossFlow", "challenge/src"])

# Project imports
from challenge.src.eval import visualize_retrieval
from eval.metrics import mrr, recall_at_k, ndcg
from common.utils import load_data, prepare_train_data, generate_submission

# Clear existing handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

In [ ]:
# Configuration dictionary
CFG = {
    "MODEL_PATH": "models/contrastive_projection.pth",
    "EPOCHS": 60,
    "BATCH_SIZE": 512,
    "LR": 5e-4,
    "TEMPERATURE": 0.05,
    "HIDDEN_DIM": 3072,
    "DROPOUT": 0.15,
    "VALIDATION_MODE": "repo_metrics",
    "MODEL": "mlp_projection",  # or "linear_projection"
    "LOSS": "contrastive",
    "SEED": 42,
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

def set_seed(seed=42):
    """Ensure deterministic reproducibility."""
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(CFG["SEED"])

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
# ============================================================================
# MODELS
# ============================================================================

class LinearProjection(nn.Module):
    """Simple linear projection with normalization."""
    def __init__(self, input_dim, output_dim, **kwargs):
        super().__init__()
        self.projection = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = F.normalize(x, dim=-1)
        out = self.projection(x)
        return F.normalize(out, dim=-1)


class MLPProjection(nn.Module):
    """Multi-layer projection with residual connections."""
    def __init__(self, input_dim, output_dim, hidden_dim=2048, dropout=0.1):
        super().__init__()
        # Bloc 1
        self.block1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        self.shortcut1 = nn.Linear(input_dim, hidden_dim) if input_dim != hidden_dim else nn.Identity()

        # Bloc 2
        self.block2 = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim)
        )

        # Optional: shortcut connection if dims match
        self.shortcut2 = nn.Linear(hidden_dim, output_dim) if hidden_dim != output_dim else nn.Identity()

        self.residual_scale = 0.5

    def forward(self, x):
        x = F.normalize(x, dim=-1)
        h = self.block1(x) + self.residual_scale * self.shortcut1(x)
        out = self.block2(h) + self.residual_scale * self.shortcut2(h)

        #out = self.net(x) + 0.1 * self.shortcut(x)  # residual with scaling
        return F.normalize(out, dim=-1)


def init_weights(m):
    """Xavier initialization for all Linear layers."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)


MODEL_REGISTRY = {
    "linear_projection": LinearProjection,
    "mlp_projection": MLPProjection
}

In [ ]:

# ============================================================================
# LOSSES
# ============================================================================

class ContrastiveLoss:
    """CLIP-style contrastive loss - FIXED VERSION."""
    def __init__(self, temperature=0.07):
        self.temperature = temperature
        # Make logit_scale learnable and properly initialized
        self.logit_scale = nn.Parameter(torch.tensor(np.log(1/temperature)))

    def __call__(self, text_features, image_features, labels=None):
        """
        Args:
            text_features: [batch_size, dim]
            image_features: [batch_size, dim]
            labels: Not used (assumes paired data)
        """
        device = text_features.device

        # Normalize (defensive, models already normalize)
        text_features = F.normalize(text_features, dim=-1)
        image_features = F.normalize(image_features, dim=-1)

        # Compute similarity with learned temperature
        logit_scale = self.logit_scale.exp().clamp(max=100)
        logits = logit_scale * (text_features @ image_features.T)

        # Create labels (diagonal = positive pairs)
        batch_size = text_features.shape[0]
        targets = torch.arange(batch_size, device=device)

        # Symmetric loss
        loss_t2i = F.cross_entropy(logits, targets)
        loss_i2t = F.cross_entropy(logits.T, targets)

        return (loss_t2i + loss_i2t) / 2


class InfoNCELoss:
    """
    InfoNCE loss with support for multiple positives per anchor.
    Better for your case where 5 captions → 1 image.
    """
    def __init__(self, temperature=0.07):
        self.temperature = temperature

    def __call__(self, text_features, image_features, labels):
        """
        Args:
            text_features: [batch_size, dim]
            image_features: [batch_size, dim]
            labels: [batch_size, batch_size] - binary matrix of matches
        """
        # Normalize
        text_features = F.normalize(text_features, dim=-1)
        image_features = F.normalize(image_features, dim=-1)

        # Similarity matrix
        logits = torch.matmul(text_features, image_features.T) / self.temperature

        # Create target distribution (can have multiple positives)
        # labels[i, j] = 1 if text i matches image j
        targets = labels.float()
        targets = targets / targets.sum(dim=1, keepdim=True).clamp(min=1e-8)

        # Cross entropy with soft targets
        log_probs = F.log_softmax(logits, dim=1)
        loss = -(targets * log_probs).sum(dim=1).mean()

        return loss


LOSS_REGISTRY = {
    "contrastive": lambda temp: ContrastiveLoss(temperature=temp),
    "infonce": lambda temp: InfoNCELoss(temperature=temp)
}

In [ ]:
# ============================================================================
# VALIDATION
# ============================================================================

@torch.inference_mode()
def validate_repo_metrics(model, val_loader, val_gallery, val_labels):
    """Evaluate using official retrieval metrics."""
    model.eval()
    preds = []
    for X, _ in tqdm(val_loader, desc="[Val]", leave=False):
        pred = model(X.to(CFG["DEVICE"]))
        preds.append(pred)
    preds = torch.cat(preds)
    return evaluate_retrieval(preds, val_gallery.to(CFG["DEVICE"]), val_labels, max_indices=100)


@torch.inference_mode()
def evaluate_retrieval(translated_embd, image_embd, gt_indices, max_indices=99, batch_size=100):
    """GPU-safe retrieval evaluation."""
    if isinstance(translated_embd, np.ndarray):
        translated_embd = torch.from_numpy(translated_embd).float()
    if isinstance(image_embd, np.ndarray):
        image_embd = torch.from_numpy(image_embd).float()

    n_queries = translated_embd.shape[0]
    all_sorted_indices = []

    for start_idx in range(0, n_queries, batch_size):
        batch_slice = slice(start_idx, min(start_idx + batch_size, n_queries))
        batch_translated = translated_embd[batch_slice]
        batch_similarity = batch_translated @ image_embd.T
        batch_indices = batch_similarity.topk(k=max_indices, dim=1, sorted=True).indices.cpu().numpy()
        all_sorted_indices.append(batch_indices)

    sorted_indices = np.concatenate(all_sorted_indices, axis=0)

    metrics = {
        'mrr': mrr,
        'ndcg': ndcg,
        'recall_at_1': lambda preds, gt: recall_at_k(preds, gt, 1),
        'recall_at_3': lambda preds, gt: recall_at_k(preds, gt, 3),
        'recall_at_5': lambda preds, gt: recall_at_k(preds, gt, 5),
        'recall_at_10': lambda preds, gt: recall_at_k(preds, gt, 10),
        'recall_at_50': lambda preds, gt: recall_at_k(preds, gt, 50),
    }

    results = {name: func(sorted_indices, gt_indices) for name, func in metrics.items()}
    return results


VALIDATION_REGISTRY = {"repo_metrics": validate_repo_metrics}

In [ ]:
# ============================================================================
# TRAINING
# ============================================================================

def train_model(model, train_loader, val_loader, val_gallery, val_labels, loss_fn, cfg):
    """Train the projection model with contrastive learning."""
    device = cfg["DEVICE"]
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg["LR"], weight_decay=1e-5)

    # Cosine annealing with warmup
    steps_per_epoch = len(train_loader)
    total_steps = steps_per_epoch * cfg["EPOCHS"]
    warmup_steps = steps_per_epoch * 2

    def lr_lambda(step):
        if step < warmup_steps:
            return step / warmup_steps
        progress = (step - warmup_steps) / (total_steps - warmup_steps)
        return 0.5 * (1 + np.cos(np.pi * progress))

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    validate = VALIDATION_REGISTRY[cfg["VALIDATION_MODE"]]
    best_metric = 0.0
    global_step = 0

    for epoch in range(cfg["EPOCHS"]):
        model.train()
        total_loss = 0

        pbar = tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}/{cfg['EPOCHS']}")
        for X, y in pbar:
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()
            pred = model(X)

            # For simple contrastive, we assume batch pairs (X[i] matches y[i])
            loss = loss_fn(pred, y, labels=None)  # ← UNE SEULE LIGNE


            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            global_step += 1

            # Update progress bar
            pbar.set_postfix({"loss": f"{loss.item():.4f}", "lr": f"{scheduler.get_last_lr()[0]:.6f}"})

        avg_loss = total_loss / len(train_loader)
        tqdm.write(f"\n> Epoch {epoch+1}: Avg Loss = {avg_loss:.6f}")

        # Validation
        val_results = validate(model, val_loader, val_gallery, val_labels)
        metric = val_results["mrr"]

        tqdm.write(f"> MRR = {metric:.4f}, R@1 = {val_results['recall_at_1']:.4f}, "
                   f"R@5 = {val_results['recall_at_5']:.4f}, R@10 = {val_results['recall_at_10']:.4f}")

        if metric > best_metric:
            best_metric = metric
            Path(cfg["MODEL_PATH"]).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), cfg["MODEL_PATH"])
            tqdm.write(f"✓ New best model saved (MRR={best_metric:.4f})\n")

    return best_metric


In [ ]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Load and prepare data
    train_data = load_data("data/train/train/train.npz")
    X, y, label = prepare_train_data(train_data)

    # Train/val split (90/10)
    perm = torch.randperm(len(X))
    split_idx = int(0.9 * len(X))
    mask = torch.zeros(len(X), dtype=torch.bool)
    mask[perm[:split_idx]] = True

    X_train, X_val = X[mask], X[~mask]
    y_train, y_val = y[mask], y[~mask]

    train_loader = DataLoader(
        TensorDataset(X_train, y_train),
        batch_size=CFG["BATCH_SIZE"],
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        TensorDataset(X_val, y_val),
        batch_size=CFG["BATCH_SIZE"],
        num_workers=4,
        pin_memory=True
    )

    # Prepare validation gallery
    val_gallery = torch.from_numpy(train_data["images/embeddings"])
    val_labels = np.nonzero(train_data["captions/label"][~mask])[1]

    # Build model
    model = MODEL_REGISTRY[CFG["MODEL"]](
        input_dim=X_train.shape[-1],
        output_dim=y_train.shape[-1],
        hidden_dim=CFG.get("HIDDEN_DIM", 2048),
        dropout=CFG.get("DROPOUT", 0.1)
    ).to(CFG["DEVICE"])

    model.apply(init_weights)

    logging.info(f"Model: {CFG['MODEL']}")
    logging.info(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Initialize loss
    loss_fn = LOSS_REGISTRY[CFG["LOSS"]](CFG["TEMPERATURE"])

    # Train
    best_metric = train_model(model, train_loader, val_loader, val_gallery, val_labels, loss_fn, CFG)
    logging.info(f"\n### Best MRR: {best_metric:.4f} ###")

    # Visualize results
    val_texts = train_data["captions/text"][~mask]
    val_caption_labels = train_data["captions/label"][~mask]  # [n_val_captions, n_images]

    # Use full gallery (no filtering)
    all_img_names = train_data["images/names"]
    all_img_embeds = torch.from_

(125000,)
Train data: 125000 captions, 125000 images


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Model: mlp_projection
Parameters: 15,750,144


[Train] Epoch 1/60:   0%|          | 0/220 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[Train] Epoch 1/60: 100%|██████████| 220/220 [02:45<00:00,  1.33it/s, loss=2.5027, lr=0.000250]



> Epoch 1: Avg Loss = 3.873774


...

> MRR = 0.2462, R@1 = 0.1523, R@5 = 0.3384, R@10 = 0.4405


[Train] Epoch 60/60: 100%|██████████| 220/220 [02:43<00:00,  1.35it/s, loss=0.5818, lr=0.000000]



> Epoch 60: Avg Loss = 0.676229


> MRR = 0.2463, R@1 = 0.1525, R@5 = 0.3385, R@10 = 0.4402

### Best MRR: 0.2472 ###


AttributeError: module 'torch' has no attribute 'from_'